In [1]:


import random                                                              
                                                                           
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.display import display, HTML
import seaborn as sns
import time

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler


CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
ratings = pd.read_csv(r'data/ratings.csv')
books = pd.read_csv(r'data/books.csv')

In [3]:
books_cleaned = books[['id','book_id','title','average_rating','work_ratings_count','original_publication_year']].rename(columns={'book_id' : 'goodreads_book_id', 'id' : 'book_id'})
display(books_cleaned);

,book_id,goodreads_book_id,title,average_rating,work_ratings_count,original_publication_year
0,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,4942365,2008.0
1,2,3,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,4800065,1997.0
2,3,41865,"Twilight (Twilight, #1)",3.57,3916824,2005.0
3,4,2657,To Kill a Mockingbird,4.25,3340896,1960.0
4,5,4671,The Great Gatsby,3.89,2773745,1925.0
...,...,...,...,...,...,...
9995,9996,7130616,"Bayou Moon (The Edge, #2)",4.09,18856,2010.0
9996,9997,208324,"Means of Ascent (The Years of Lyndon Johnson, #2)",4.25,12952,1990.0
9997,9998,77431,The Mauritius Command,4.35,10733,1977.0
9998,9999,8565083,Cinderella Ate My Daughter: Dispatches from th...,3.65,11994,2011.0


In [4]:
# compute weighted ranking scores for each book

r = books_cleaned['average_rating'] # average rating per book
v = books_cleaned['work_ratings_count'] # number of ratings per book
c = books_cleaned['average_rating'].mean() # average rating across entire dataset
m = books_cleaned['work_ratings_count'].quantile(.90) # top 10% of books with the most ratings

books_weighted_ranking = books_cleaned[['book_id','title','average_rating','work_ratings_count']].copy()
books_weighted_ranking['weighted_score'] = (r*v + c*m) / (v+m)

# create table that sorts books based on ranking
books_weighted_ranking = books_weighted_ranking.sort_values(by=['weighted_score'], ascending=False)

In [5]:
# function that returns most popular books

def popular_rec(num_books):
    return books_weighted_ranking[['title']][0:num_books]
popular_rec(10)

,title
24,Harry Potter and the Deathly Hallows (Harry Po...
26,Harry Potter and the Half-Blood Prince (Harry ...
17,Harry Potter and the Prisoner of Azkaban (Harr...
23,Harry Potter and the Goblet of Fire (Harry Pot...
421,"Harry Potter Boxset (Harry Potter, #1-7)"
134,"A Storm of Swords (A Song of Ice and Fire, #3)"
191,The Name of the Wind (The Kingkiller Chronicle...
20,Harry Potter and the Order of the Phoenix (Har...
1,Harry Potter and the Sorcerer's Stone (Harry P...
160,"The Return of the King (The Lord of the Rings,..."


In [6]:
import pickle
pickle_out = open("popular.pkl","wb")
pickle.dump(popular_rec,pickle_out)
pickle_out.close()